## Importing Dataset Cleaned by Matchms and Lookups

In [ ]:
import os
import sys
import gensim
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matchms import Scores, Spectrum
from matchms.importing import load_from_json



In [ ]:
ROOT = os.path.dirname(os.getcwd())
#path_data = os.path.join(ROOT, 'data')
path_data = 'C:\\Users\\User\\Data'
sys.path.insert(0, ROOT)

from matchms.importing import load_from_json

filename = os.path.join(path_data,'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.json')
spectrums = load_from_json(filename)

print("number of spectra:", len(spectrums))


## Plotting peaks per spectrum

In [ ]:
number_of_peaks = [len(spec.peaks) for spec in spectrums]


plt.figure(figsize=(12,7))
hist = plt.hist(number_of_peaks, np.arange(0,2000,20))
plt.xlabel("number of peaks in spectrum")
plt.ylabel("number of spectra in respective bin")

## Post-Process

Preparing data for cosine similarity scoring. Done by:

- normalize peaks (maximum intensity to 1)
- remove peaks outside [0, 1000] m/z window
- remove spectra with < 10 peaks
- remove peaks with intensities < 0.01 of maximum intensity.

In [ ]:
from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses
from matchms.filtering import reduce_to_number_of_peaks

def post_process(s):
    s = normalize_intensities(s)
    s = select_by_mz(s, mz_from=0, mz_to=1000)
    s = select_by_relative_intensity(s, intensity_from=0.01, intensity_to=1.0)
    s = reduce_to_number_of_peaks(s, 10, 150, None)
    return s

# apply filters to the data
spectrums = [post_process(s) for s in spectrums]

# omit spectrums that didn't qualify for analysis
spectrums = [s for s in spectrums if s is not None]


print("Remaining spectra after post process", len(spectrums))



In [ ]:
spectrumsblah = spectrums.copy()



In [ ]:
spectrumswithpeak = []

def get_parent_peak(s): 
    precursor = s.metadata['precursor_mz']  
    tolerance = (precursor / 1000000) * 5
    condition = (s.peaks.mz >= precursor - tolerance) & (s.peaks.mz <= precursor + tolerance)
    mz_candidates = s.peaks.mz[condition]
    intensities_candidates = s.peaks.intensities[condition]
    if len(mz_candidates) is 0:
        return 0
    else: 
        max_index = np.argmax(intensities_candidates)
        return mz_candidates[max_index], intensities_candidates[max_index]

for spec in spectrums:
    if(get_parent_peak(spec) != 0):
        spectrumswithpeak.append(spec)
    
print(len(spectrumswithpeak))
    

In [ ]:
def find_chem_string(spec):
    s = spec.metadata['inchi']
    if(s == "") or (s is None):
        return None
    else:
        string_in_slashes = s.split('/')[1].strip()
        return string_in_slashes
    
def are_peaks_similar(mass1, mass2):
    tolerance = 1
    if(-1 <= (mass1 - mass2) <= 1):
        return True
    else:
        return False
    
    


In [ ]:
spectrums_with_inchi = []

for spec in spectrumswithpeak:
    if(find_chem_string(spec) is not None):
        spectrums_with_inchi.append(spec)


In [ ]:
print(len(spectrums_with_inchi))

print(find_chem_string(spectrums_with_inchi[0]))

print(find_chem_string(spectrums_with_inchi[1]))

print(find_chem_string(spectrums_with_inchi[2]))

print(find_chem_string(spectrums_with_inchi[3]))

print(find_chem_string(spectrums_with_inchi[4]))

print(find_chem_string(spectrums_with_inchi[4]))

print(find_chem_string(spectrums_with_inchi[11]))

In [ ]:

def is_molecule_here(spec, spec_list):
    ishere = 0
    mcl = find_chem_string(spec)
    print(mcl)
    for s in spec_list:
        lib_mcl = find_chem_string(s)
        if(mcl == lib_mcl) and (are_peaks_similar(spec, s) == True):
            ishere = 1
    return ishere

library_spectrums = []
query_spectrums = []

for spec in spectrums_with_inchi:    
        if(is_molecule_here(spec,library_spectrums) == 0) & (len(library_spectrums) < 3000):
            library_spectrums.append(spec)

        elif(is_molecule_here(spec,query_spectrums) == 0) & (len(query_spectrums) < 1000):
            query_spectrums.append(spec)

        print("library is ", len(library_spectrums), "and query is ", len(query_spectrums))
  
          
print(len(library_spectrums))
print(len(query_spectrums))


molecule_matches = 0
for spec in query_spectrums:
    if(is_molecule_here(spec, library_spectrums) == True):
        molecule_matches += 1

print("Molecule matches =", molecule_matches)

    
    

In [ ]:
safety_library = library_spectrums.copy()
safety_query = query_spectrums.copy()

In [ ]:
def look_for_inchi_and_precursor(query, library):
    queries_with_inchi_match = []
    queries_with_precursor_match = []
    queries_with_inchi_match_and_precursor = []
    
    for spec in query:
        for s in library:
            if(find_chem_string(spec) == find_chem_string(s)):
                queries_with_inchi_match.append(spec)          
            if(are_peaks_similar(spec.metadata['precursor_mz'], s.metadata['precursor_mz']) == True):
                queries_with_precursor_match.append(spec)
            if(find_chem_string(spec) == find_chem_string(s)) and ((are_peaks_similar(spec.metadata['precursor_mz'], s.metadata['precursor_mz']) == True)):
                    queries_with_inchi_match_and_precursor.append(spec)
                    
    print(len(queries_with_inchi_match))  
    print(len(queries_with_precursor_match))      
    print(len(queries_with_inchi_match_and_precursor))      
    type_of_match = ["Inchi_Match", "Precurs_Match", "Inchi_&_Precurs_Match"]
    num_of_matches= [len(queries_with_inchi_match), len(queries_with_precursor_match), len(queries_with_inchi_match_and_precursor)]
                    
    fig = plt.figure(figsize=(12,7))
    ax = fig.add_axes([0,0,1,1])
    ax.bar(type_of_match, num_of_matches)
    plt.xlabel("number of peaks in spectrum")
    plt.ylabel("number of spectra in respective bin")
    plt.show()
    

    
look_for_inchi_and_precursor(query_spectrums, library_spectrums)
                
        
        
        
        



In [ ]:
def analyse_spectrums(list):  

    number_of_peaks = [len(spec.peaks) for spec in list]
    plt.figure(figsize=(12,7))
    hist = plt.hist(number_of_peaks, np.arange(0,2000,20))
    plt.xlabel("number of peaks in spectrum")
    plt.ylabel("number of spectra in respective bin")
    average = 0
    min = len(list[0].peaks.mz)
    max  = 0
    min_spec = None
    max_spec = None
    print(min)
    for spec in list:
        average += len(spec.peaks.mz) 
        if min > len(spec.peaks.mz):
            min = len(spec.peaks.mz)
            min_spec = spec
        if max < len(spec.peaks.mz):
            max = len(spec.peaks.mz)
            max_spec = spec
    average = average / len(list)
    print(average)
    print(min)
    min_spec.plot()
    print(max)
    max_spec.plot()
    
analyse_spectrums(library_spectrums)
analyse_spectrums(query_spectrums)


In [ ]:
from numpy.random import seed
from numpy.random import rand




class fragment_peak:
    mz = 0
    intensity = 0
    spectrum_id = ""
    
    def __init__(self, mz, intensity, spectrum_id):
        self.mz = mz
        self.intensity = intensity
        self.spectrum_id = spectrum_id   
    
    def __lt__(self, other):
        return self.mz < other.mz

    def _eq_(self, other):
        return self.mz == other.mz
    
    def print_fragment(self):
        print("Peak with mz: ", self.mz, ", intensity: ", self.intensity, ", from spectrum: ", self.spectrum_id)
   
    

frag1 = fragment_peak(spectrums[3].peaks.mz[1], spectrums[3].peaks.intensities[1], spectrums[3].get('spectrum_id'))
frag2 = fragment_peak(spectrums[1].peaks.mz[1], spectrums[1].peaks.intensities[1], spectrums[1].get('spectrum_id'))
frag3 = fragment_peak(spectrums[2].peaks.mz[1], spectrums[2].peaks.intensities[1], spectrums[2].get('spectrum_id'))
frag4 = fragment_peak(spectrums[5].peaks.mz[1], spectrums[5].peaks.intensities[1], spectrums[5].get('spectrum_id'))
frag5 = fragment_peak(spectrums[6].peaks.mz[1], spectrums[6].peaks.intensities[1], spectrums[6].get('spectrum_id'))

frag_list = [frag1, frag2, frag3, frag4, frag5]
frag_order = [f.mz for f in frag_list]

print(frag_order)

frag_list_sorted = sorted(frag_list)
frag_list_sorted_order = [f.mz for f in frag_list_sorted]

print(frag_list_sorted_order)





In [ ]:
import bisect
import operator

def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

def find_lt(a, x):
    'Find rightmost value less than x'
    i = bisect.bisect_left(a, x)
    if i:
        return a[i-1]
    raise ValueError

def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect.bisect_right(a, x)
    if i:
        return a[i-1]
    raise ValueError

def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect.bisect_right(a, x)
    if i != len(a):
        return a[i]
    raise ValueError

def find_ge(a, x):
    'Find leftmost item greater than or equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a):
        return a[i]
    raise ValueError
    
def find_frags_in_range(a, x, y):  
    i = bisect.bisect_left(a, x)
    j = bisect.bisect_right(a,y)
    return a[i:j]

def find_randomfrags_in_spec(fraglist, id): 
    indices = [i for i, x in enumerate(fraglist) if x.spectrum_id == id]
    frags = [fraglist[i] for i in indices]
    randomfrags = random.sample(frags, 5)
    return randomfrags
    
    
    
   # indices = [i for i, x in enumerate(fraglist) if x.spectrum_id == id]
   # print("Indices are: ", indices)
   #  print(type(indices[0]))
   # print(type(indices[2]))
   # fragments = fraglist[indices] frags = operator.itemgetter(*fraglist)(indices)
   # randomfragments = random.sample(fragments, 5)
   # return randomfragments
    
    
    
    

In [ ]:
All_fragments_list = []

j = 0;

for spec in library_spectrums :
    i = 0;
    for p in spec.peaks.mz:
        frag = fragment_peak(spec.peaks.mz[i], spec.peaks.intensities[i], spec.get('spectrum_id'))
        i += 1
        All_fragments_list.append(frag)  
    print("Full spectrum turned to ", i, " fragments ")
    print(len(All_fragments_list), " frags in list")
    print(j, " of ", len(library_spectrums), " spectrums turned to fragment objects.")
    j += 1   
    
    


In [ ]:
print("Unsorted fragments lists:")

for i in range(20):
    All_fragments_list[i].print_fragment()
    
all_fragments_list_sorted = sorted(All_fragments_list)

print("Sorted by MZs: ")

for i in range(20):
    all_fragments_list_sorted[i].print_fragment()
    


In [ ]:
number_of_peaks = [len(spec.peaks) for spec in spectrums]

print(spectrums[1].peaks.intensities)
print(spectrums[2].peaks.intensities)

plt.figure(figsize=(12,7))
hist = plt.hist(number_of_peaks, np.arange(0,2000,20))
plt.xlabel("number of peaks in spectrum")
plt.ylabel("number of spectra in respective bin")

## Naive Decoy Approach

For the naive decoy spectral library, we use all possible fragment ions from the reference library of spectra and then randomly add these ions to the decoy spectra library, until each decoy spectrum reaches the desired number of fragment ions that mimics the corresponding library spectrum. This method is presented as a baseline evaluation of the other, more intricate methods.

1. Determine if there is a parent peak in target spectrum
2. If parent peak = false, move to next spectrum in target spectra list.
3. If parent peak = true, add to new decoy object:
   - The parent peak is added to the array of peaks in the new decoy object
4. Find number of peaks in target spectrum (the length of the list containing all peaks)
5. Select that number of spectra randomly from the list of all spectra in target list
6. Select from each of those individually a random peak
7. Add those peaks to the list of peaks in the decoy spectrum, until number peaks in decoy spectrum = number of peaks in target spectrum. 
8. Add new complete decoy spectrum to decoy spectra library (naive)



public Peak getParentPeak(Spectrum s) {
        Peak parentpeak;
        ArrayList<Peak> candidatepeaks = new ArrayList<Peak>;
        Peak p = s.getPrecursor;
        for(Peak counterpeak : s.getPeaks()) {
            if(counterpeak.getMass >= p.getMass - 5 && counterpeak.getMass <= p.getMass + 5) {
               candidatepeaks.add(counterpeak);
               }
            }  
         if(candidatepeaks.size() == 0) {
            system.out.print("No parent peak found")
                                                                                      
            }
         else {                                                                   
               for(Peak p : candidatepeaks) {
                  if(p.getIntensity() > parentpeak.getIntensity()) {
                     parentpeak = p
                    }
         return parentpeak; 
    
  
    
                                                                                    
                                                                                  
 public spectrum createDecoy(Spectrum s) {  
         parent = getParentPeak(s);
         if(parent == none) {
                system.out.print("No parent peak detected: decoy not created.")
                return null;
                 }
         else {
                 Spectrum decoy = new Spectrum();
                 decoy.peaks.addPeak(parent);
                 Int peaksrequired = s.peaks.size() - 1;
                 for(int i = 0; i < peaksrequired; i++) {
                     ArrayList<Peak> possiblepeaks = new ArrayList<Peak>;                             
                     Random r = new Random() (range 0 to spectrums.size());
                     possiblepeaks = spectrums(r).getPeaks();
                     Random r2 = new Random() (range 0 to possiblepeaks.size());
                     decoy.peaks.addPeak(possiblepeaks(r2));
                    }
                  return decoy; 
                     
                     
              
         
         

In [ ]:
from matchms.Spikes import Spikes
import random



def get_spectrum_info(s):
    print(s.peaks.mz)
    precursor = s.metadata['precursor_mz']
    print("Precursor spectrum is: ", precursor)
    print("Number of  peaks in spectrum is: ", len(s.peaks) )
    masses = s.peaks.mz
    print(masses)
    print(s.peaks.intensities)     
    mz = s.peaks.mz
    intensities = s.peaks.intensities
    print(intensities)
    s.plot()


def ispeakhere(s, precursor): 
    tolerance = (precursor / 1000000) * 5
    return np.any((s.peaks.mz >= precursor - 5) & (s.peaks.mz <= precursor + 5))                                         


def masswithin5ppm(m,range):
    ishere = 0
    tolerance = (m / 1000000) * 5
    for r in range:
        if m >= r - tolerance and m <= r + tolerance:
            ishere = 1
            break
    return ishere
    
    








In [ ]:
naive_decoy_spectrums = []



def create_naive_decoy(s):
    print(get_parent_peak(s))
    decoy_mz = np.array([get_parent_peak(s)[0]])
    decoy_intensity = np.array([get_parent_peak(s)[1]])                        
    peaks_in_target = len(s.peaks.mz)

    random_spectrums = random.sample(library_spectrums, peaks_in_target - 1)

    for spec in random_spectrums:
        randommass =  random.choice(spec.peaks.mz)
        index = np.where(spec.peaks.mz == randommass)
        randomintensity = spec.peaks.intensities[index]
        decoy_mz = np.append(decoy_mz, [randommass])
        decoy_intensity = np.append(decoy_intensity, [randomintensity])
        


    decoy_mz = np.asarray(decoy_mz, dtype=float) 
    decoy_intensity = np.asarray(decoy_intensity, dtype=float) 

    inds  = decoy_mz.argsort()

    sorted_intensities = decoy_intensity[inds]
    sorted_mzs = decoy_mz[inds]

    decoy = Spectrum(sorted_mzs, sorted_intensities) 

    naive_decoy_spectrums.append(decoy)
       

                            
print( "Total processed peaks = ", len(naive_decoy_spectrums))

i = 1

for spec in library_spectrums:
    create_naive_decoy(spec)
    print(i, " naive decoy created")
    i += 1


print( "Total processed peaks = ", len(naive_decoy_spectrums))

# print(condition)

## Spectrum-based Decoy Approach

The second method is similar to the naive method, as we create the decoy spectral library through choosing fragment ions that co-appear in the spectra from the target spectral library (Fig. 1c): In this spectrum-based approach, we start with an empty set of fragment ion candidates. First, the precursor fragment ion of the target spectrum is added to the decoy spectrum. For each fragment ion added to the decoy spectrum, we choose all spectra from the target spectral library which contain this fragment ion, within a mass range of 5 p.p.m. From these spectra, we uniformly draw (all fragment ions have the same probability to be drawn) five fragment ions that are added to the fragment ion candidate set; we use all fragment ions in case there are fewer than five. We draw a fragment ion from the fragment ion candidate set and add it to the decoy spectrum, then proceed as described above until we reach the desired number of fragment ions that mimics the corresponding library spectrum. The two-step process of first drawing candidates, then drawing the actual decoy spectrum was introduced to better mimic fragmentation cascades and dependencies between fragments. Furthermore, it prevents that fragment-rich spectra dominate the process. Out of the five added candidate fragment ions, between zero and five end up in the final decoy spectrum. Fragment ions with mass close (5 p.p.m.) to a previously added fragment ion mass, or masses above the precursor fragment ion mass are discarded. If the precursor ion is absent from the MS/MS spectrum, we use the selected ion mass to find matching compound masses. 

1. Determine if there is a parent peak in target spectrum
2. If parent peak = false, move to next spectrum in target spectra list.
3. If parent peak = true, add to new decoy spectrum object:
   - The parent peak is added to the array of peaks in the new decoy object
4. Create a list of all spectra from the target list that contain the same fragment ion (search all peaks for a mass identical to or within a close range of the parent peak - mass range of 5 p.p.m)
5. Create an empty list of candidate fragment ions that will be drawn from
6. Draw 5 fragment ions from each of the spectra on list created at step 4 and add those to candidate list created at step 5
7. Randomly draw a fragment ion from the candidate list:
   - If its mass < precursor peak of target spectrum AND is not within 5 p.p.m of any other peak, then the peak is added to the      decoy spectrum's list of peaks.
   - If it fails either of the conditions above, it is discarded and another candidate fragment is drawn.
8. Repeat step 4- 7 until number peaks in decoy spectrum = number of peaks in target spectrum. 
9. Add completed decoy to decoy spectral library. 


In [ ]:
def random_sample_5_peaks(spectrum_id):
    frag_list =[x for x in All_fragments_list if x.spectrum_id == spectrum_id]
    frag_list = random.sample(frag_list, 5)
    return frag_list


In [ ]:
def get_spectrums_with_peak(mz):
    tolerance = (mz / 1000000) * 5
    x = fragment_peak(mz - tolerance, 0, 'counter')
    y = fragment_peak(mz + tolerance, 0, 'counter2')
    x = find_ge(all_fragments_list_sorted, x)
    y = find_le(all_fragments_list_sorted, y)
    
   # print("Mass for search is:", mz)
    
    first_frags = find_frags_in_range(all_fragments_list_sorted, x, y)
    
    # print("In first frags there are: ", len(first_frags), "there masses are ")   

    # for f in first_frags:
     #   print(f.mz)
        
  #  print("Now retrieving spectrums:")
        
    first_spectrum_ids = [f.spectrum_id for f in first_frags]
    
   # print("unsorted spectrum ids: ", len(first_spectrum_ids))
    
    final_spectrum_ids = list(dict.fromkeys(first_spectrum_ids))
    
    if(len(final_spectrum_ids) > 25):
        final_spectrum_ids = random.sample(final_spectrum_ids, 25)
        print("Too large: random sampled 25")
    
    
    #print("de-duplicated spectrum ids", len(final_spectrum_ids))
    
          
    return final_spectrum_ids



def return_random_pick(candidatefrags,decoypeaks, parentmass):
    
   # print("Number of candidates on arrival: ", len(candidatefrags)) 
    candidatefrags = [x for x in candidatefrags if x.mz < parentmass] 
    
   #  print("Number of candidates after those under parent peak thrown:", len(candidatefrags))
    
    candidatefrags2 =[ x for x in candidatefrags if masswithin5ppm(x.mz, decoypeaks) == 0]
 
    
    try:
        candidateion = random.choice(candidatefrags2)
        
    except: 
        candidateion = random.choice(candidatefrags)
           
    return candidateion
          
    
    
    
    
    
    
    

In [ ]:


def create_spectrum_based_decoy_bisect(s):
   # print("This spectrum has: ", len(s.peaks.mz), " peaks.")
    parentmass = get_parent_peak(s)[0]
    parentintensity = get_parent_peak(s)[1]
    decoy_mz = np.array([parentmass])
    decoy_intensities = np.array([parentintensity])
   # print("Parent peak equals: ", parentmass, "m/z, with intensity: ", parentintensity)
    peaks_in_target = len(s.peaks.mz)  

    candidate_fragments_list = []
    
    mass_for_loop_seeding = parentmass.copy()
    
    while(len(decoy_mz) < len(s.peaks.mz)):
    
        id_list = get_spectrums_with_peak(mass_for_loop_seeding)

        for id in id_list:
            random_peaks = random_sample_5_peaks(id)
            candidate_fragments_list.extend(random_peaks)
            
        print("Length of candidate frags list: ", len(candidate_fragments_list))

        drawn_ion = return_random_pick(candidate_fragments_list, decoy_mz, parentmass)

        print("Drew randomly:", drawn_ion.mz)
        
    

        decoy_mz = np.append(decoy_mz, drawn_ion.mz)
        decoy_intensities = np.append(decoy_intensities, drawn_ion.intensity)
        
        
        print("Added peak with mass ", drawn_ion.mz, "and intensity ", drawn_ion.intensity)
        print("Decoy mz is length ", len(decoy_mz))
        
        mass_for_loop_seeding = drawn_ion.mz
    
    
        
    
    print("Decoy masses has this number: ", len(decoy_mz))
    print("Decoy intensities has this number: ", len(decoy_intensities)) 

                
    decoy_mz = np.asarray(decoy_mz, dtype=float) 
    decoy_intensities = np.asarray(decoy_intensities, dtype=float) 
    inds  = decoy_mz.argsort()

    sorted_intensities = decoy_intensities[inds]
    sorted_masses = decoy_mz[inds]

    decoy = Spectrum(sorted_masses, sorted_intensities) 
    

    complex_decoy_spectrums.append(decoy)

     

        
            
            
    
    

In [ ]:
import time

decoyscreated = 1

times_taken = []

complex_decoy_spectrums = []


for spec in library_spectrums:
    start = time.time()
    create_spectrum_based_decoy_bisect(spec)
    end = time.time()
    print("-----------------------------------------------------------------------------------------------------")
    print(" Decoy", decoyscreated, "of", len(library_spectrums), "created. Took ", end - start, "to do.")
    print("------------------------------------------------------------------------------------------------------")
    decoyscreated += 1
    end = time.time()
    timetaken = end-start
    times_taken.append(timetaken)
    print("Time for decoy with", len(spec.peaks.mz), " peaks: ", timetaken)
   
    


In [ ]:
from matchms.exporting import save_as_mgf


np.save('library_spectrums_21August', library_spectrums)

In [ ]:
from matchms.similarity import CosineGreedy

class CosineHit:   
    score = 0
    type = None
    query = None
    library = None
    qvalue = None
    
    def __init__(self, score, type, query, library):
        self.score = score
        self.type = type
        self.query = query
        self.library =library
        
    def __lt__(self, other):
        return self.score < other.score

    def _eq_(self, other):
        return self.score == other.score
    
    def getQuery(self):
        return self.query
    
    def getLibrary(self):
        return self.library
    
    def getQvalue(self):
        return self.qvalue
    
    def setValue(self, q):
        self.qvalue = q
        
    def setType(Self, type):
        self.type = type

def plot_hits(cosine_hits):
    
    scores = [score in cosine_hit in cosine_hits] 
    plt.figure(figsize=(12,7))
    hist = plt.hist(scores, np.arange(0,1, 0.05))
    plt.xlabel("cosine score of matches")
    plt.ylabel("number of queries in respective bin")



def return_list_cosine_scores(query, library, librarytype):
         
        if(librarytype != "library" and librarytype != "decoy"):
            print("library type paramter must be either target or decoy")
            return False
        else: 
            cosine_greedy = CosineGreedy(tolerance=0.2)
            counter = 1
            scores = []
            average_matches = 0
            milestone = 1
            
            if(librarytype == "decoy"):        
                for spec in query:
                    prelim_scores = []
                    for d in library:
                        score, n_matches = cosine_greedy(d, spec)
                        average_matches = average_matches + n_matches
                        newscore = CosineHit(score, librarytype, spec, d)
                        prelim_scores.append(newscore)          

                    prelim_scores = sorted(prelim_scores)
                    scores.append(prelim_scores[-1])
                    print(milestone)
                    milestone += 1
                    
            if(librarytype == "library"):
                for spec in query:
                    prelim_scores = []
                    for d in library:
                        if(are_peaks_similar(spec.metadata['precursor_mz'], d.metadata['precursor_mz']) == True):
                            score, n_matches = cosine_greedy(d, spec)
                            average_matches = average_matches + n_matches
                            newscore = CosineHit(score, librarytype, spec, d)
                            prelim_scores.append(newscore)          

                    prelim_scores = sorted(prelim_scores)
                    scores.append(prelim_scores[-1])
                    print(milestone)
                    milestone += 1

            return scores


    

    

In [ ]:
scores_library = return_list_cosine_scores(query_spectrums, library_spectrums, "library")

In [ ]:
print(len(scores_library))

for score in scores_library[-20:]:
    print(score.score)
    print(score.query.metadata['inchi'], score.library.metadata['inchi'])

scores_library_sorted = sorted(scores_library)

for score in scores_library_sorted[-20:]:
    print(score.score)


In [ ]:
scores_complex = return_list_cosine_scores(query_spectrums, complex_decoy_spectrums, "decoy")
scores_naive= return_list_cosine_scores(query_spectrums, naive_decoy_spectrums, "decoy")

print(len(scores_complex))

for score in scores_complex[-20:]:
    print(score.score)

scores_complex_decoys_sorted = sorted(scores_complex)
scores_naive_decoys_sorted = sorted(scores_naive)

for score in scores_complex_decoys_sorted[-20:]:
    print(score.score)


In [ ]:

def create_fdr_list_decoys(sorted_scores): 
    fdr_at_index = []
    for score in sorted_scores:
        list_to_consider = [x for x in sorted_scores if x.score >= score.score]
        decoy_hits_in_list = 0
        library_hits_in_list = 0
        for s in list_to_consider:     
            if(s.spectrumtype == "library"):
                library_hits_in_list += 1
            else:
                decoy_hits_in_list +=1
        fdr = decoy_hits_in_list / library_hits_in_list
        fdr_at_index.append(fdr)
    
    print(fdr_at_index)
    return fdr_at_index

def create_fdr_list_for_trues(sorted_scores):   
    fdr_at_index = []
    for score in sorted_scores:
        list_to_consider = [x for x in sorted_scores if x.score >= score.score]
        false_hits_in_list = 0
        true_hits_in_list = 0
        for s in list_to_consider:     
            if(find_chem_string(s.getQuery()) == find_chem_string(s.getLibrary())):
                true_hits_in_list += 1
            else:
                false_hits_in_list +=1             
        fdr = false_hits_in_list / true_hits_in_list
        fdr_at_index.append(fdr)
        print("Found fdr ", fdr, "for score ", score.score)
        
    return fdr_at_index


def enumerate_reversed(seq):
    n = len(seq)
    for obj in reversed(seq):
        n -= 1
        yield n, obj

def create_q_values_from_fdr_list(fdrlist, scores):
    print(len(fdrlist))
    for num, score in enumerate_reversed(scores):
        if(score.spectrumtype == "library") or (score.spectrumtype == "true"):
            print(num)
            if(num == 1999):
                qvalue = fdrlist[-1]
            else: 
                list_of_fdr_to_consider = fdrlist[ 0 : num]
                qvalue = min(list_of_fdr_to_consider)   
                print("For score", score.score, " qvalue is: ", qvalue )
            score.setValue(qvalue)
                      
    scores_to_return = [x for x in scores if x.spectrumtype == "library"] 
    return scores_to_return

def create_q_values_from_fdr_list_true(fdrlist, scores):
    print(len(fdrlist))
    for num, score in enumerate_reversed(scores):
        if(find_chem_string(score.getQuery()) == find_chem_string(score.getLibrary())):
            print(num)
            if(num == len(fdrlist) -1):
                qvalue = fdrlist[-1]
            else: 
                list_of_fdr_to_consider = fdrlist[ 0 : num]
                qvalue = min(list_of_fdr_to_consider)   
                print("For score", score.score, " qvalue is: ", qvalue )
            score.setValue(qvalue)
                      
    scores_to_return = [x for x in scores if x.spectrumtype == "library"] 
    return scores_to_return
    
    


    
    


In [ ]:
scores_complex_merged = scores_library + scores_complex
scores_complex_merged = sorted(scores_complex_merged)
scores_complex_fdr_list = create_fdr_list_decoys(scores_complex_merged)
print(len(scores_complex_fdr_list))
complex_q_values = create_q_values_from_fdr_list(scores_complex_fdr_list, scores_complex_merged)
print(len(complex_q_values))


In [ ]:
true_scores_fdr_list = create_fdr_list_for_trues(scores_library)
print(true_scores_fdr_list)
true_qvalues = create_q_values_from_fdr_list_true(true_scores_fdr_list, scores_library)

In [ ]:
truescores = []
falsescores = []

for score in scores_library:
    if(find_chem_string(score.getQuery()) == find_chem_string(score.getLibrary())):
        truescores.append(score)
    else:
        falsescores.append(score)
        
        
plottrues = []
plotfalses = []

for s in truescores:
    plottrues.append(s.score)
    
for s in falsescores:
    plotfalses.append(s.score)


print(len(scores_library))
print(len(plottrues))
print(len(plotfalses))


plt.figure(figsize=(12,7))
hist = plt.hist(plotfalses, np.arange(0,1,0.05))
plt.xlabel("cosine score of scores")
plt.ylabel("number of scores in respective bin")
        

In [ ]:
qvalues_true = []

qvalues_complex_estimate = []

qvalues_naive_estimate = []

for s in qvalues_library:
    qvalues_true.append(s.qvalue)
        
for s in qvalue_estimates_complex_decoys:
    qvalues_complex_estimate.append(s.qvalue)
        
for s in qvalue_estimates_naive_decoys: 
    qvalues_naive_estimate.append(s.qvalue)
        
        
print(len(qvalues_true))

print(len(qvalues_complex_estimate))

print(len(qvalues_naive_estimate))
    
          
plt.figure(figsize=(17,7))
plt.title("title")
plt.xlabel( 'True Qvalues')
plt.ylabel('Estimated Qvalues')
plt.plot(qvalues_complex_estimate, qvalues_true, 'rx', label = "Spectrum-Based Decoys")
plt.plot(qvalues_naive_estimate, qvalues_true, 'bx', label = "Naive Decoys")
plt.legend()
plt.show()


